In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import mplhep as hep
hep.style.use('ATLAS')
import os
from tqdm import tqdm
import h5py


### 根据 particle list文件计算pT谱

In [2]:
def calculate_rapidity(E, pz):
    """计算快度 y = 0.5*ln((E+pz)/(E-pz))"""
    with np.errstate(divide='ignore', invalid='ignore'):
        y = 0.5 * np.log((E + pz) / (E - pz))
        return np.nan_to_num(y, posinf=10, neginf=-10)

def calculate_pseudorapidity(px, py, pz):
    """计算赝快度 η = -ln[tan(θ/2)] = 0.5*ln((p+pz)/(p-pz))"""
    p = np.sqrt(px**2 + py**2 + pz**2)
    with np.errstate(divide='ignore', invalid='ignore'):
        eta = 0.5 * np.log((p + pz) / (p - pz))
        return np.nan_to_num(eta, posinf=10, neginf=-10)

def load_particle_data(base_path, n_files=1):
    """加载多个粒子数据文件并合并"""
    particle_lists = []
    for i in range(n_files):
        file_path = os.path.join(base_path, f'mc_particle_list{i}')
        if os.path.exists(file_path):
            data = np.loadtxt(file_path)
            particle_lists.append(data)
        else:
            print(f"Warning: File not found - {file_path}")
    if particle_lists:
        return np.vstack(particle_lists)
    else:
        raise FileNotFoundError("未找到任何有效文件！")


In [3]:
# def compute_pt_spectrum(base_path, pid=211, kind='Eta', 
#                       rapidity_window=1.0, nsampling=4000, pt_range=(0, 5), pt_bins=50):
#     """
#     计算 pT 谱 可以使用不同的截断方式（ y/Eta ）

#     参数:
#     - base_path: 数据文件路径（如 `/path/to/mc_particle_list/`）
#     - pid: 粒子 PDG 编码（默认 211，即 π⁺）
#     - kind: 使用 'Y'（快度）或 'Eta'（赝快度，默认）
#     - rapidity_window: 快度窗口（默认 1.0，即 |y|<0.5）
#     - nsampling: 抽样次数（默认 4000）
#     - pt_range: pT 范围（默认 0-5 GeV）
#     - pt_bins: pT 分 bin 数量（默认 50）

#     返回:
#     - res: 数组，形状 (n_bins, 2)，包含 [pT, dN/(2πpT dpT dy)]
#     """
#     # 加载数据
#     particle_lists = load_particle_data(base_path, n_files=nsampling)
#     particle_lists = particle_lists[particle_lists[:, -1] != 0]  # 过滤无效粒子,保留带电粒子

#     # 定义列索引
#     MOM_X0, MOM_X1, MOM_X2, MOM_X3, PDGCODE = 5, 6, 7, 8, 9

#     # 提取基本物理量
#     E = particle_lists[:, MOM_X0]
#     px, py, pz = particle_lists[:, MOM_X1], particle_lists[:, MOM_X2], particle_lists[:, MOM_X3]
#     pdg = particle_lists[:, PDGCODE].astype(int)

#     # 计算快度/赝快度
#     y = calculate_rapidity(E, pz)
#     eta = calculate_pseudorapidity(px, py, pz)

#     # 合并数据
#     dat = np.column_stack((E, px, py, pz, y, pdg, eta))

#     # 选择粒子类型
#     if pid == 'charged':
#         particle_type = (dat[:, 5] == dat[:, 5])  # 所有带电粒子
#     else:
#         particle_type = (dat[:, 5] == pid)  # 特定 PDG 编码粒子

#     # 提取快度/赝快度区间的粒子
#     rapidity_col = 6 if kind == 'Eta' else 4
#     Yi = dat[particle_type, rapidity_col]
#     #     print(Yi.max()-Yi.min())

#     # 计算 pT
#     pti = np.sqrt(dat[particle_type, 1]**2 + dat[particle_type, 2]**2)
#     pti = pti[np.abs(Yi) < 0.5 * rapidity_window]  # 快度窗口选择

#     # 计算 pT 谱
#     dN, pt_edges = np.histogram(pti, bins=pt_bins,range=pt_range)
#     dpt = pt_edges[1:] - pt_edges[:-1]
#     pt_centers = 0.5 * (pt_edges[1:] + pt_edges[:-1])

#     # 归一化
#     dNdpt = dN / (2 * np.pi * float(nsampling) * pt_centers * dpt * rapidity_window)

#     return np.column_stack((pt_centers, dNdpt))

In [4]:
def compute_pt_spectrum(base_path, pid=211, kind='Eta', 
                        rapidity_window=1.0, nsampling=4000, pt_range=(0, 5), pt_bins=50):
    """
    计算 pT 谱（使用η截断，但用y范围归一化）,Kind只选择“Eta”
    
    参数:
    - base_path: 数据文件路径
    - pid: 粒子 PDG 编码（默认 211，π⁺）
    - kind: 必须为 'Eta'（此版本强制用η截断）
    - rapidity_window: η截断窗口（默认 1.0，即 |η|<0.5）
    - nsampling: 抽样次数（默认 4000）
    - pt_range: pT 范围（默认 0-5 GeV）
    - pt_bins: pT 分 bin 数量（默认 50）
    
    返回:
    - res: 数组 [pT, dN/(2πpT dpT dy)]
    - y_eff: 实际快度(y)范围（用于归一化分母）
    """
    # 加载数据
    particle_lists = load_particle_data(base_path, n_files=nsampling)
    particle_lists = particle_lists[particle_lists[:, -1] != 0]  # 过滤无效粒子,保留带电粒子

    # 定义列索引
    MOM_X0, MOM_X1, MOM_X2, MOM_X3, PDGCODE = 5, 6, 7, 8, 9

    # 提取基本物理量
    E = particle_lists[:, MOM_X0]
    px, py, pz = particle_lists[:, MOM_X1], particle_lists[:, MOM_X2], particle_lists[:, MOM_X3]
    pdg = particle_lists[:, PDGCODE].astype(int)

    # 计算快度/赝快度
    y = calculate_rapidity(E, pz)
    eta = calculate_pseudorapidity(px, py, pz)
    
    # 合并数据
    dat = np.column_stack((E, px, py, pz, y, pdg, eta))
    
    # 选择粒子类型
    if pid == 'charged':
        particle_type = (dat[:, 5] == dat[:, 5])  # 所有带电粒子
    else:
        particle_type = (dat[:, 5] == pid)  # 特定 PDG 编码粒子
    # 选择粒子类型

    # 提取快度/赝快度区间的粒子
    rapidity_col = 6 if kind == 'Eta' else 4
    Yi = dat[particle_type, rapidity_col]

    # 计算 pT
    pti = np.sqrt(dat[particle_type, 1]**2 + dat[particle_type, 2]**2)
    Y_cut = np.abs(Yi) < 0.5 * rapidity_window
    pti = pti[Y_cut]  # 快度窗口选择

    # 计算 pT 谱
    dN, pt_edges = np.histogram(pti, bins=pt_bins, range=pt_range)
    dpt = pt_edges[1:] - pt_edges[:-1]
    pt_centers = 0.5 * (pt_edges[1:] + pt_edges[:-1])

    
    # 获取截断后的粒子的快度(y)范围
    y_selected = dat[particle_type, 4][Y_cut]  # y列
    y_min, y_max = np.min(y_selected), np.max(y_selected)
    y_eff = y_max - y_min  # 实际快度范围
    print(y_eff)

    # 用实际快度范围归一化
    dNdpt = dN / (2 * np.pi * float(nsampling) * pt_centers * dpt * y_eff)

    return np.column_stack((pt_centers, dNdpt))

In [5]:
def compute_average_pt_spectrum(base_dir, pid=211, kind='Eta', 
                               rapidity_window=1.0, nsampling=4000, 
                               pt_range=(0, 5), pt_bins=50, num_events=1000):
    """
    计算多个事件的平均pT谱
    
    参数:
    - base_dir: 基础目录路径（包含event0到event999子目录）
    - pid: 粒子PDG编码
    - kind: 'Eta'或'Y'
    - rapidity_window: 快度/赝快度窗口
    - nsampling: 抽样次数
    - pt_range: pT范围
    - pt_bins: pT分bin数量
    - num_events: 事件数量
    
    返回:
    - 平均pT谱 [pT, dN/(2πpT dpT dy)]
    """
    # 初始化累积数组
    total_spectrum = np.zeros((pt_bins, 2))
    event_count = 0
    
    # 使用 tqdm 包装循环
    for event_num in tqdm(range(num_events), desc="Processing events"):
        event_dir = os.path.join(base_dir, f"event{event_num}", "mc_particle_list/")
        
        try:
            # 计算当前事件的pT谱
            spectrum = compute_pt_spectrum(
                base_path=event_dir,
                pid=pid,
                kind=kind,
                rapidity_window=rapidity_window,
                nsampling=nsampling,
                pt_range=pt_range,
                pt_bins=pt_bins
            )
            
            # 累积结果
            total_spectrum[:, 1] += spectrum[:, 1]
            event_count += 1
            
        except Exception as e:
            print(f"\nError processing event {event_num}: {str(e)}")  # 换行避免干扰 tqdm
            continue
    
    # 计算平均值
    if event_count > 0:
        total_spectrum[:, 1] /= event_count
        print(f"\nSuccessfully processed {event_count}/{num_events} events.")  # 换行避免干扰 tqdm
    else:
        print("\nNo events were processed successfully.")
    
    # 设置pT中心值（所有事件相同）
    pt_edges = np.linspace(pt_range[0], pt_range[1], pt_bins + 1)
    total_spectrum[:, 0] = 0.5 * (pt_edges[1:] + pt_edges[:-1])
    
    return total_spectrum

### 给定中心度的给定事件，计算4000次采样的平均结果

In [11]:
# 使用示例
average_pt_spectrum = compute_average_pt_spectrum(
#     base_dir='/media/mengzhu/74e6f585-9c67-40c0-a5e9-89fb98e77bab/home/mengzhu/d_Au_work/d_Au_DWF/d_Au_work_pT_spectra/hydro_result_n_BC/Tfrz_128Mev/ebe_mc_nsampling_4000/plat_1p3_gw_1p3/entropy3D_y0_4p0_sigma_2p5_L_factor_2p0_R_factor_2p0_P_factor_5p5_alpha_1p0_beta_0p35/dAu200/0_20',
    base_dir="/media/mengzhu/disk2T/d_Au_work/d_Au_DWF/d_Au_work_pT_spectra/hydro_result_n_BC/" \
            "/Tfrz_180Mev_ETAOS_0p16_tau0p6/ebe_mc_nsampling_4000/plat_1p3_gw_1p3/" \
            "/entropy3D_y0_4p0_sigma_2p5_L_factor_2p0_R_factor_2p0_P_factor_5p5_alpha_1p0_beta_0p35/dAu200/40_60",
    pid=211,
    kind='Eta',
    rapidity_window=0.7,
    nsampling=4000,
    pt_range=(0, 5),
    pt_bins=50,
    num_events=1
)

Processing events: 100%|██████████████████████████| 1/1 [00:00<00:00,  2.73it/s]

0.6871319739687995

Successfully processed 1/1 events.


### 进行并行计算

In [13]:
from concurrent.futures import ProcessPoolExecutor, as_completed

def compute_average_pt_spectrum(base_dir, pid=211, kind='Eta', 
                               rapidity_window=1.0, nsampling=4000, 
                               pt_range=(0, 5), pt_bins=50, num_events=1000):
    """
    计算多个事件的平均pT谱
    
    参数:
    - base_dir: 基础目录路径（包含event0到event999子目录）
    - pid: 粒子PDG编码
    - kind: 'Eta'或'Y'
    - rapidity_window: 快度/赝快度窗口
    - nsampling: 抽样次数
    - pt_range: pT范围
    - pt_bins: pT分bin数量
    - num_events: 事件数量
    
    返回:
    - 平均pT谱 [pT, dN/(2πpT dpT dy)]
    """
    # 初始化累积数组
    total_spectrum = np.zeros((pt_bins, 2))
    event_count = 0
    
    # 使用 tqdm 包装循环
    for event_num in tqdm(range(num_events), desc="Processing events"):
        event_dir = os.path.join(base_dir, f"event{event_num}", "mc_particle_list/")
        
        try:
            # 计算当前事件的pT谱
            spectrum = compute_pt_spectrum(
                base_path=event_dir,
                pid=pid,
                kind=kind,
                rapidity_window=rapidity_window,
                nsampling=nsampling,
                pt_range=pt_range,
                pt_bins=pt_bins
            )
            
            # 累积结果
            total_spectrum[:, 1] += spectrum[:, 1]
            event_count += 1
            
        except Exception as e:
            print(f"\nError processing event {event_num}: {str(e)}")  # 换行避免干扰 tqdm
            continue
    
    # 计算平均值
    if event_count > 0:
        total_spectrum[:, 1] /= event_count
        print(f"\nSuccessfully processed {event_count}/{num_events} events.")  # 换行避免干扰 tqdm
    else:
        print("\nNo events were processed successfully.")
    
    # 设置pT中心值（所有事件相同）
    pt_edges = np.linspace(pt_range[0], pt_range[1], pt_bins + 1)
    total_spectrum[:, 0] = 0.5 * (pt_edges[1:] + pt_edges[:-1])
    
    return total_spectrum

def process_single_event(args):
    """处理单个事件的辅助函数（用于并行化）"""
    event_num, base_dir, kwargs = args
    event_dir = os.path.join(base_dir, f"event{event_num}", "mc_particle_list/")
    
    try:
        spectrum = compute_pt_spectrum(base_path=event_dir, **kwargs)
        return spectrum[:, 1]  # 只返回dNdpt部分
    except Exception as e:
        print(f"\nError in event {event_num}: {str(e)}")
        return None

def compute_average_pt_spectrum_parallel(base_dir, pid=211, kind='Eta',
                                       rapidity_window=1.0, nsampling=4000,
                                       pt_range=(0, 5), pt_bins=50, 
                                       num_events=1000, workers=4):
    """
    并行计算平均pT谱
    
    参数:
    - workers: 并行进程数（建议设置为CPU核心数）
    """
    # 初始化结果数组
    pt_edges = np.linspace(pt_range[0], pt_range[1], pt_bins + 1)
    pt_centers = 0.5 * (pt_edges[1:] + pt_edges[:-1])
    total_spectrum = np.zeros((pt_bins, 2))
    total_spectrum[:, 0] = pt_centers
    
    # 准备参数
    kwargs = {
        'pid': pid,
        'kind': kind,
        'rapidity_window': rapidity_window,
        'nsampling': nsampling,
        'pt_range': pt_range,
        'pt_bins': pt_bins
    }
    
    # 并行处理
    with ProcessPoolExecutor(max_workers=workers) as executor:
        # 准备任务列表
        tasks = [(i, base_dir, kwargs) for i in range(num_events)]
        
        # 提交任务并使用tqdm显示进度
        futures = [executor.submit(process_single_event, task) for task in tasks]
        successful_events = 0
        
        for future in tqdm(as_completed(futures), total=num_events, 
                          desc=f"Processing {num_events} events"):
            result = future.result()
            if result is not None:
                total_spectrum[:, 1] += result
                successful_events += 1
    
    # 计算平均值
    if successful_events > 0:
        print(f'suuccessful event number={successful_events}')
        total_spectrum[:, 1] /= successful_events
        print(f"\nSuccessfully processed {successful_events}/{num_events} events")
    else:
        raise RuntimeError("No events processed successfully")
    
    return total_spectrum

def compute_all_centralities_parallel(base_template, centralities, 
                                    num_events_dict, workers=4, **kwargs):
    """
    并行计算所有中心度的pT谱
    
    参数:
    - base_template: 路径模板，如 '/path/to/dAu200/{centrality}'
    - centralities: 中心度列表 ['0_20', '20_40', ...]
    - num_events_dict: 每个中心度对应的事件数字典 {'0_20': 500, ...}
    - workers: 总并行进程数（会在中心度和事件两级分配）
    """
    results = {}
    workers_per_centrality = max(1, workers // len(centralities))
    
    with tqdm(centralities, desc="Centralities") as pbar:
        for centrality in pbar:
            pbar.set_postfix({'centrality': centrality})
            base_dir = base_template.format(centrality=centrality)
            
            results[centrality] = compute_average_pt_spectrum_parallel(
                base_dir=base_dir,
                num_events=num_events_dict.get(centrality, 1000),
                workers=workers_per_centrality,
                **kwargs
            )
    
    return results


### 输入中心度，以及中心度中的事件数量，计算各个中心度的pT结果

In [14]:
# 配置参数
centralities = ['0_20', '20_40', '40_60', '60_88']
# centralities = ['0_20']

num_events_dict = {'0_20': 1000, '20_40': 1000, '40_60': 1000, '60_88': 1000}  # 示例值
# base_template = '/media/mengzhu/74e6f585-9c67-40c0-a5e9-89fb98e77bab/home/mengzhu/d_Au_work/d_Au_DWF/d_Au_work_pT_spectra/hydro_result_n_BC/Tfrz_128Mev/ebe_mc_nsampling_4000/plat_1p3_gw_1p3/entropy3D_y0_4p0_sigma_2p5_L_factor_2p0_R_factor_2p0_P_factor_5p5_alpha_1p0_beta_0p35/dAu200/{centrality}'  # 替换为实际路径
base_template = "/media/mengzhu/disk2T/d_Au_work/d_Au_DWF/d_Au_work_pT_spectra/hydro_result_n_BC/"\
"/Tfrz_150Mev_ETAOS_0p16_tau0p6/ebe_mc_nsampling_4000/plat_1p3_gw_1p3/"\
"/entropy3D_y0_4p0_sigma_2p5_L_factor_2p0_R_factor_2p0_P_factor_5p5_alpha_1p0_beta_0p35/dAu200/{centrality}"  




In [15]:
# 运行计算
pT_spectra_211 = compute_all_centralities_parallel(
    base_template=base_template,
    centralities=centralities,
    num_events_dict=num_events_dict,
    pid=211,
    kind='Eta',
    rapidity_window=0.7,
    nsampling=4000,
    pt_range=(0, 3),
    pt_bins=30,
    workers=16  # 总并行进程数
)



Processing 1000 events:   0%|                          | 0/1000 [00:00<?, ?it/s]

0.6964741576511513



Processing 1000 events:   0%|                  | 1/1000 [00:02<35:45,  2.15s/it]

0.6927714379656607
0.6913490198737227
0.6952593999764183



Processing 1000 events:   0%|                  | 4/1000 [00:02<07:14,  2.29it/s]

...

0.6888656709060974



Processing 1000 events: 100%|███████████████▉| 997/1000 [02:38<00:00,  5.48it/s]

0.6912352951946852
0.6897185435134634
0.6920523353858765



Centralities: 100%|████████████| 4/4 [21:16<00:00, 319.07s/it, centrality=60_88]

suuccessful event number=1000

Successfully processed 1000/1000 events


In [16]:
# 运行计算
pT_spectra_2212 = compute_all_centralities_parallel(
    base_template=base_template,
    centralities=centralities,
    num_events_dict=num_events_dict,
    pid=2212,
    kind='Eta',
    rapidity_window=0.7,
    nsampling=4000,
    pt_range=(0, 3),
    pt_bins=30,
    workers=16  # 总并行进程数
)



Processing 1000 events:   0%|                          | 0/1000 [00:00<?, ?it/s]

0.5734464496306502



Processing 1000 events:   0%|                  | 1/1000 [00:01<30:38,  1.84s/it]

0.58353034010418940.5876696220842804




Processing 1000 events:   0%|                  | 2/1000 [00:01<13:37,  1.22it/s]

...

0.5654341969256507
0.5580730685853936



Processing 1000 events: 100%|███████████████▉| 998/1000 [02:38<00:00,  6.40it/s]

0.5487948941435874
0.5082921374548023



Centralities: 100%|████████████| 4/4 [21:05<00:00, 316.48s/it, centrality=60_88]

suuccessful event number=1000

Successfully processed 1000/1000 events


In [17]:
# 运行计算
pT_spectra_321 = compute_all_centralities_parallel(
    base_template=base_template,
    centralities=centralities,
    num_events_dict=num_events_dict,
    pid=321,
    kind='Eta',
    rapidity_window=0.7,
    nsampling=4000,
    pt_range=(0, 3),
    pt_bins=30,
    workers=16  # 总并行进程数
)

Processing 1000 events:   0%|                          | 0/1000 [00:00<?, ?it/s]

0.6648054436152446



Processing 1000 events:   0%|                  | 1/1000 [00:01<32:01,  1.92s/it]

0.6540808743151116
0.6695764380017649
0.6632799100954118



Processing 1000 events:   0%|                  | 4/1000 [00:02<07:15,  2.29it/s]

...

0.6250806172519751



Processing 1000 events: 100%|███████████████▉| 997/1000 [02:38<00:00,  5.60it/s]

0.6172646988170092
0.6220929338398252
0.6332690952215059



Centralities: 100%|████████████| 4/4 [21:15<00:00, 318.75s/it, centrality=60_88]

suuccessful event number=1000

Successfully processed 1000/1000 events


### 保存pt spectra数据

In [18]:
import h5py

# Save
with h5py.File('./data/pt_spectra/pT_spectra_211_Tfrz_150Mev_ETAOS_0p16_EtaCut_tau0p6.h5', 'w') as f:
    for key, value in pT_spectra_211.items():
        f.create_dataset(key, data=value)
with h5py.File('./data/pt_spectra/pT_spectra_321_Tfrz_150Mev_ETAOS_0p16_EtaCut_tau0p6.h5', 'w') as f:
    for key, value in pT_spectra_321.items():
        f.create_dataset(key, data=value)
with h5py.File('./data/pt_spectra/pT_spectra_2212_Tfrz_150Mev_ETAOS_0p16_EtaCut_tau0p6.h5', 'w') as f:
    for key, value in pT_spectra_2212.items():
        f.create_dataset(key, data=value)


